In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

#Estimate the ratio of single and non-single populations in Japan
path1_pattern = ''
path2_pattern = ''

years = range(2011, 2023)

total_population = [
    127798704, 127515133, 127297686, 127234652,127094745 ,127041812,
    126918546, 126748506, 126555078, 126146099,
    125502290, 124946789
]
single_population = [
    17111190.0, 17437873.0, 17764556.0, 18091239.0, 18417922, 18964546.0,
    19511170.0, 20057794.0, 20604418.0, 21151042, 21697666.0, 22244290.0
]

output_path = ''

with pd.ExcelWriter(output_path) as writer:
    for index, year in enumerate(years):
        path1 = path1_pattern.format(year)
        path2 = path2_pattern.format(year)

        data1 = pd.read_excel(path1, sheet_name='household', usecols=[0, 1], index_col=0)
        data2 = pd.read_excel(path2, sheet_name='single', usecols=[0, 1], index_col=0)
        combined = data1.join(data2, lsuffix='_non_single', rsuffix='_single')
        non_single_population = total_population[index] - single_population[index]
        combined['expenditure_non_single'] = combined['expenditure_non_single'] * non_single_population
        combined['expenditure_single'] = combined['expenditure_single'] * single_population[index]
        combined['ratio_household'] = combined['expenditure_non_single'] / (combined['expenditure_non_single']+combined['expenditure_single'])        
        combined['ratio_single'] = combined['expenditure_single'] / (combined['expenditure_non_single']+combined['expenditure_single'])
        combined.to_excel(writer, sheet_name=str(year))



ratio = df['single'] / df['household']
average_ratio = ratio.mean()

ssp = pd.read_excel('SSP_single.xlsx')

for scenario in ['ssp1', 'ssp2', 'ssp3', 'ssp4', 'ssp5']:
    ssp[scenario + '_single'] = ssp[scenario + '_household'] * average_ratio

# If desired, the carbon footprint can be scaled so that the total carbon footprint equals total emissions in 2050

for scenario in ['ssp1', 'ssp2', 'ssp3', 'ssp4', 'ssp5']:
    total = ssp[scenario + '_single'] + ssp[scenario + '_household']
    scale = total_emission / total
    ssp[scenario + '_single'] *= scale
    ssp[scenario + '_household'] *= scale

# Read data
pop_df = pd.read_excel('')
carbon_df = pd.read_excel('')
ssp_df = pd.read_excel('') 
forecast_df = pd.read_excel('') 

# Calculate the per capita carbon emissions of single and non-single people in 2005, 2011 and 2015
for year in [2005, 2011, 2015]:
    pop = pop_df[pop_df['Year'] == year]
    carbon_single = carbon_df.loc[year, 'single']
    carbon_household = carbon_df.loc[year, 'household']
    per_capita_single = carbon_single / pop['Estimated_Single_Population']
    per_capita_household = carbon_household / (pop['Total_Population'] - pop['Estimated_Single_Population'])

# Establish a linear regression model to predict future per capita carbon emissions
model_single = LinearRegression()
model_household = LinearRegression()
model_single.fit(np.array(carbon_df.index).reshape(-1, 1), carbon_df['single'])
model_household.fit(np.array(carbon_df.index).reshape(-1, 1), carbon_df['household'])

future_years = np.array([2030, 2035, 2040]).reshape(-1, 1)
forecast_single = model_single.predict(future_years)
forecast_household = model_household.predict(future_years)
# Use the total carbon emissions given by the SSP scenario and multiply them by the predicted per capita carbon emissions of single and non-single people to obtain the future carbon emissions of single and non-single people.
ssp_single = ssp_df * forecast_single
ssp_household = ssp_df * forecast_household
